In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True) 

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
# split into training set and test
# in regression we skip this step in gridsearch method, but in classification we do this in order to get the confusion matrix and classification report.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid = {'penalty':['l1', 'l2', 'elasticnet', None],
              'random_state':[0, 10,100], 'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']}

grid = GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


C:\Users\Default.DESKTOP-73FCCDO\Documents\Installations\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
165 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Default.DESKTOP-73FCCDO\Documents\Installations\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Default.DESKTOP-73FCCDO\Documents\Installations\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Defa

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet', None],
                         'random_state': [0, 10, 100],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [14]:
# print best parameter after tuning
# print(grid.best_params_)
clf=grid.cv_results_
grid_predictions=grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, grid_predictions)

# print classification report
from sklearn.metrics import classification_report
clf_report=classification_report(y_test, grid_predictions)

In [15]:
# f1 score is one of the evaluation metrics
# sample to check f1 macro value
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l1', 'random_state': 0, 'solver': 'saga'}: 0.8794289739855382


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [10 39]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [18]:
# roc_auc score is also an evaluation metric (a scoring parametr in classification)
# if the roc_auc value is nearly 1, then the model performs well

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9493397358943578

In [19]:
table=pd.DataFrame.from_dict(clf)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_random_state,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001132,0.001386,0.000000,0.000000,l1,0,lbfgs,"{'penalty': 'l1', 'random_state': 0, 'solver':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
1,0.002439,0.001081,0.003608,0.001145,l1,0,liblinear,"{'penalty': 'l1', 'random_state': 0, 'solver':...",0.804764,0.811321,0.66593,0.92351,0.922185,0.825542,0.094918,37
2,0.001817,0.000995,0.000000,0.000000,l1,0,newton-cg,"{'penalty': 'l1', 'random_state': 0, 'solver':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
3,0.000401,0.000803,0.000000,0.000000,l1,0,newton-cholesky,"{'penalty': 'l1', 'random_state': 0, 'solver':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
4,0.001205,0.000984,0.000000,0.000000,l1,0,sag,"{'penalty': 'l1', 'random_state': 0, 'solver':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.001005,0.000898,0.000000,0.000000,None,100,liblinear,"{'penalty': None, 'random_state': 100, 'solver...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
68,0.003537,0.000961,0.002935,0.001279,None,100,newton-cg,"{'penalty': None, 'random_state': 100, 'solver...",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
69,0.003236,0.002035,0.001959,0.001404,None,100,newton-cholesky,"{'penalty': None, 'random_state': 100, 'solver...",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
70,0.003908,0.000950,0.002511,0.000778,None,100,sag,"{'penalty': None, 'random_state': 100, 'solver...",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1


In [20]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 15
Salary: 5000
Sex Male 0 or 1: 0


In [21]:
Future_Prediction = grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
